<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1


    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance


    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-31 15:16:16
-------------------
qualified stocks: 95
with latest results: 30
still star stocks: 20
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  4.51 L
Current:  1.37 C
-------------------
Today PnL: -80.16 K (-0.58%)
Current PnL: -17.73 L (-12.06%)
CY Booked + Current PnL: -9.70 L (-6.6%)
-------------------
Total profit:  3.17 L
Total loss:  -20.90 L
-------------------
Total Booked + Current PnL: 17.28 L (14.4%)
Total Booked PnL: 35.01 L (29.16%)
Curr Year Booked PnL: 8.03 L (5.87%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.13 C
Est FTT PnL: 75.95 L (55.48%)
Deployed:  1.20 C
Current:  1.37 C
CAGR/XIRR %: 8.69%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
4,APOLLOHOSP,6904.43,8285.00,-17.52,H-LC,6.59,112358.0,8792.0,11921.0,0.54,8.49,10.61,20.00,15.0,0.74,0.85,23.76,X40N,BTT,HEALTHCARE
17,BRITANNIA,5190.70,6446.05,15.59,H-LC,9.67,103734.0,10301.0,12292.0,0.28,11.03,11.85,24.18,31.0,0.84,0.78,25.96,XY25,ATH,FMCG
39,HINDALCO,651.95,761.55,-20.73,H-LC,5.26,109224.0,4912.0,12626.0,-0.89,4.71,11.56,16.81,10.0,0.39,0.83,21.51,X5K,ATH,METALS
74,SIS,477.00,477.00,2344.56,H-SC,6.61,65764.0,-15326.0,15323.0,0.69,-18.90,23.30,0.00,157.0,-1.00,0.50,33.28,OX40N,NTT,MISC
29,DIXON,14367.00,18940.15,-26.82,H-MC,5.81,134712.0,19776.0,16812.0,0.44,17.21,12.48,31.83,85.0,1.18,1.02,51.61,X40N,ATH,IT


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,QUESS,296.83,986.00,-34.99,M-SC,41.84,64496.0,-510.0,151437.0,-1.11,-0.78,234.80,232.18,199.0,-0.00,0.49,0.20,XY24,NTT,MISC
3,ALKYLAMINE,2347.98,4546.37,8.61,H-SC,19.80,101097.0,134.0,94394.0,3.99,0.13,93.37,93.63,149.0,0.00,0.76,50.58,SR,ATH,CHEMICALS
14,BERGEPAINT,561.33,680.00,-19.20,H-MC,10.39,228622.0,1283.0,46776.0,-1.82,0.56,20.46,21.14,107.0,0.03,1.73,28.00,XY24,NTT,PAINTS


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,CIPLA,1492.70,1795.0,-19.10,H-LC,1.74,213035.0,8535.0,32871.0,-0.28,4.17,15.43,20.25,9.0,0.26,1.61,13.27,X40N,BTT,PHARMA
10,BAJAJHFL,122.26,152.0,-10.01,H-LC,2.50,186583.0,-13923.0,62692.0,-1.07,-6.94,33.60,24.33,30.0,-0.22,1.41,7.59,X40N,BTT,FINANCE
56,LTIM,5564.16,7230.2,-4.50,H-LC,2.68,183978.0,-16332.0,76314.0,-0.56,-8.15,41.48,29.94,24.0,-0.21,1.39,27.10,X200,ATH,IT
5,ASIANPAINT,2961.56,4250.0,-17.94,H-LC,3.23,162901.0,-38485.0,126102.0,-0.84,-19.11,77.41,43.51,19.0,-0.31,1.23,13.40,X40,BTT,PAINTS
40,HINDUNILVR,2413.81,2723.0,-9.90,H-LC,4.23,282262.0,11915.0,22722.0,3.40,4.41,8.05,12.81,28.0,0.52,2.13,18.05,XY25,NTT,FMCG


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ALKYLAMINE,2347.98,4546.37,8.61,H-SC,19.80,101097.0,134.0,94394.0,3.99,0.13,93.37,93.63,149.0,0.00,0.76,50.58,SR,ATH,CHEMICALS
40,HINDUNILVR,2413.81,2723.00,-9.90,H-LC,4.23,282262.0,11915.0,22722.0,3.40,4.41,8.05,12.81,28.0,0.52,2.13,18.05,XY25,NTT,FMCG
51,JIOFIN,310.58,387.00,7.36,H-LC,13.68,230160.0,12754.0,40738.0,2.65,5.87,17.70,24.61,50.0,0.31,1.74,63.65,XY24,BTT,FINANCE
68,SAMMAANCAP,170.35,326.00,-165.91,M-SC,4.25,76200.0,-26010.0,119398.0,2.52,-25.45,156.69,91.37,210.0,-0.22,0.58,23.70,XY25,NTT,FINANCE
88,UNITDSPR,1376.96,1644.00,-11.95,H-LC,7.43,177025.0,-4734.0,39990.0,1.92,-2.60,22.59,19.39,11.0,-0.12,1.34,4.70,X40N,NTT,BREWERIES


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,GREENPANEL,375.16,537.00,221.06,M-SC,8.40,143204.0,-39874.0,118859.0,-8.28,-21.78,83.00,43.14,232.0,-0.34,1.08,34.70,XY24,NTT,MISC
18,BSOFT,628.85,836.99,-18.64,H-SC,12.55,68302.0,-41747.0,78172.0,-5.90,-37.93,114.45,33.10,129.0,-0.53,0.52,13.50,XR,ATH,IT
16,BLUSPRING,226.45,94.95,NaN,NaN,6.77,19027.0,-30566.0,1768.0,-5.20,-61.63,9.29,-58.07,261.0,-17.29,0.14,27.60,XY24,ATH,MISC
64,RELAXO,902.64,1176.00,-35.12,H-SC,13.73,58656.0,-56882.0,91873.0,-5.11,-49.23,156.63,30.28,134.0,-0.62,0.44,14.59,X40N,NTT,FOOTWEAR
20,CAMS,3643.00,5226.82,-6.58,H-SC,0.27,104866.0,2862.0,41485.0,-3.73,2.81,39.56,43.48,120.0,0.07,0.79,20.06,X40N,ATH,MISC


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,SYMPHONY,1306.42,1306.0,-31.46,M-SC,8.89,145921.0,-25220.0,25171.0,-2.29,-14.74,17.25,-0.03,197.0,-1.00,1.10,5.59,OX40N,NTT,DURABLES
46,INDIGOPNTS,1407.73,1408.0,127.12,H-SC,6.15,148800.0,-25759.0,25787.0,-0.23,-14.76,17.33,0.02,135.0,-1.00,1.12,29.03,OX40N,NTT,PAINTS
86,UJJIVANSFB,52.77,53.0,37.69,H-SC,20.87,119475.0,-23004.0,23620.0,1.40,-16.15,19.77,0.44,163.0,-0.97,0.90,41.01,OX40N,NTT,BANKS
80,TATAELXSI,7332.28,7332.0,-18.66,H-MC,1.84,85372.0,-17280.0,17279.0,-0.05,-16.83,20.24,-0.00,101.0,-1.00,0.64,29.78,OX40N,NTT,IT
21,CERA,7989.07,8422.0,-17.66,H-SC,2.89,117702.0,-26101.0,33898.0,0.39,-18.15,28.80,5.42,155.0,-0.77,0.89,29.89,OX40N,NTT,CERAMICS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,TCS,3794.03,4998.00,-27.38,H-LC,13.45,236824.0,-59110.0,153012.0,-0.57,-19.97,64.61,31.73,1.0,-0.39,1.79,0.00,X40,BTT,IT
32,ENRIN,1377.95,3234.40,NaN,NaN,2.89,80860.0,46411.0,0.0,0.23,134.73,0.00,134.73,2.0,inf,0.61,23.65,AR,ATH,ELECTRICAL
91,VBL,492.64,672.28,-7.32,H-LC,11.92,281105.0,16065.0,80593.0,0.08,6.06,28.67,36.46,3.0,0.20,2.12,20.05,X40N,ATH,FMCG
48,INFY,1461.46,2275.00,-15.65,H-LC,7.13,270075.0,8474.0,137144.0,-0.67,3.24,50.78,55.67,4.0,0.06,2.04,9.54,X40,BTT,IT
1,ABB,5551.76,7934.00,-39.87,H-LC,7.50,214695.0,-1824.0,94723.0,-0.88,-0.84,44.12,42.91,5.0,-0.02,1.62,12.22,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,267.99,NaN,NaN,4.12,50957.0,9702.0,7735.0,-1.93,23.52,15.18,42.26,198.0,1.25,0.38,19.97,XY24,ATH,IT
58,MEDANTA,1087.93,1486.00,18.55,H-SC,12.07,151960.0,25760.0,20423.0,0.28,20.41,13.44,36.59,156.0,1.26,1.15,35.48,XY24,NTT,HEALTHCARE
32,ENRIN,1377.95,3234.40,NaN,NaN,2.89,80860.0,46411.0,0.0,0.23,134.73,0.00,134.73,2.0,inf,0.61,23.65,AR,ATH,ELECTRICAL


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,KPIGREEN,497.21,731.64,25.90,H-SC,12.77,128835.0,3538.0,55541.0,-0.68,2.82,43.11,47.15,144.0,0.06,0.97,61.18,MH,ATH,POWER
58,MEDANTA,1087.93,1486.00,18.55,H-SC,12.07,151960.0,25760.0,20423.0,0.28,20.41,13.44,36.59,156.0,1.26,1.15,35.48,XY24,NTT,HEALTHCARE
3,ALKYLAMINE,2347.98,4546.37,8.61,H-SC,19.80,101097.0,134.0,94394.0,3.99,0.13,93.37,93.63,149.0,0.00,0.76,50.58,SR,ATH,CHEMICALS
26,DABUR,505.20,735.00,-5.92,H-MC,4.72,203580.0,6552.0,83061.0,0.71,3.33,40.80,45.49,100.0,0.08,1.54,16.80,XY24,BTT,FMCG
20,CAMS,3643.00,5226.82,-6.58,H-SC,0.27,104866.0,2862.0,41485.0,-3.73,2.81,39.56,43.48,120.0,0.07,0.79,20.06,X40N,ATH,MISC


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3234.40,NaN,NaN,2.89,80860.0,46411.0,0.0,0.23,134.73,0.00,134.73,2.0,inf,0.61,23.65,AR,ATH,ELECTRICAL
93,WHIRLPOOL,1219.98,2270.00,-40.45,M-SC,7.51,100185.0,8686.0,70069.0,-3.48,9.49,69.94,86.07,211.0,0.12,0.76,44.93,XR,NTT,DURABLES
45,INDIAMART,2327.09,4850.92,-47.33,H-SC,13.15,136941.0,13605.0,120152.0,-0.05,11.03,87.74,108.45,118.0,0.11,1.03,35.85,AR,ATH,MISC
53,KPIGREEN,497.21,731.64,25.90,H-SC,12.77,128835.0,3538.0,55541.0,-0.68,2.82,43.11,47.15,144.0,0.06,0.97,61.18,MH,ATH,POWER
94,WIPRO,243.46,420.00,-10.12,M-LC,5.55,153884.0,2939.0,106519.0,-0.80,1.95,69.22,72.51,55.0,0.03,1.16,8.09,XR,NTT,IT


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,94.95,NaN,NaN,6.77,19027.0,-30566.0,1768.0,-5.20,-61.63,9.29,-58.07,261.0,-17.29,0.14,27.60,XY24,ATH,MISC
74,SIS,477.00,477.00,2344.56,H-SC,6.61,65764.0,-15326.0,15323.0,0.69,-18.90,23.30,0.00,157.0,-1.00,0.50,33.28,OX40N,NTT,MISC
24,COFFEEDAY,59.14,80.00,-53.72,L-SC,21.07,68678.0,-44871.0,84920.0,-1.19,-39.52,123.65,35.27,267.0,-0.53,0.52,68.09,XR,NTT,HOTELS
32,ENRIN,1377.95,3234.40,NaN,NaN,2.89,80860.0,46411.0,0.0,0.23,134.73,0.00,134.73,2.0,inf,0.61,23.65,AR,ATH,ELECTRICAL
57,MASFIN,326.60,399.50,-18.70,H-SC,16.82,92280.0,-5700.0,27573.0,-0.13,-5.82,29.88,22.32,151.0,-0.21,0.70,33.84,XR,ATH,FINANCE


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,267.99,NaN,NaN,4.12,50957.0,9702.0,7735.0,-1.93,23.52,15.18,42.26,198.0,1.25,0.38,19.97,XY24,ATH,IT
58,MEDANTA,1087.93,1486.00,18.55,H-SC,12.07,151960.0,25760.0,20423.0,0.28,20.41,13.44,36.59,156.0,1.26,1.15,35.48,XY24,NTT,HEALTHCARE
16,BLUSPRING,226.45,94.95,NaN,NaN,6.77,19027.0,-30566.0,1768.0,-5.20,-61.63,9.29,-58.07,261.0,-17.29,0.14,27.60,XY24,ATH,MISC
91,VBL,492.64,672.28,-7.32,H-LC,11.92,281105.0,16065.0,80593.0,0.08,6.06,28.67,36.46,3.0,0.20,2.12,20.05,X40N,ATH,FMCG
72,SHALBY,261.39,327.00,970.18,M-SC,9.06,145447.0,-35696.0,81159.0,-0.01,-19.71,55.80,25.10,233.0,-0.44,1.10,16.56,XY24,NTT,HEALTHCARE


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.78
1,25,41.73
2,50,70.74


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    41.06
LC    35.65
MC    22.15
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.70
X40N     14.97
X40      12.66
XY25     10.93
XR        9.06
AR        8.27
OX40N     7.86
X5K       2.23
MH        1.71
X200      1.39
SR        1.21
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    27.49
H-SC    27.45
H-MC    19.20
M-SC    12.15
M-LC     7.11
M-MC     2.61
L-SC     1.46
L-LC     1.05
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.63,-0.88,34.64
IT,11.78,-16.47,73.85
FINANCE,8.42,-17.11,59.60
BANKS,7.48,-15.01,61.95
PAINTS,5.73,-15.02,37.69
ELECTRICAL,5.45,-2.02,50.56
MISC,5.32,-22.90,86.60
HEALTHCARE,4.72,3.22,22.48
AUTO,4.33,-17.77,69.01


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2786236.0,25
XR,1095456.0,13
AR,1048882.0,10
X40,777395.0,10
X40N,568447.0,12
XY25,490130.0,8
OX40N,336080.0,10
SR,182896.0,2
X5K,145085.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2687407.0,30
M-SC,1363799.0,15
H-LC,1225977.0,20
H-MC,1161292.0,15
M-LC,437781.0,5
M-MC,333484.0,2
L-SC,268681.0,3
L-MC,57091.0,1
L-LC,50283.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      926236.0      7
M-SC       XY24      840744.0      7
H-SC       AR        613095.0      4
           XR        514506.0      6
H-LC       X40       509705.0      6
H-MC       XY24      454314.0      4
M-MC       XY24      333484.0      2
H-LC       X40N      273768.0      6
H-SC       X40N      219045.0      4
H-LC       AR        218093.0      2
H-MC       XY25      208130.0      2
           X40       192636.0      3
H-SC       SR        182896.0      2
L-SC       XR        181342.0      2
M-LC       XY24      181217.0      2
H-SC       OX40N     176088.0      6
M-SC       AR        158354.0      2
M-LC       X5K       132459.0      1
H-MC       XR        121123.0      1
M-SC       XY25      119398.0      1
           XR        114875.0      2
M-LC       XR        106519.0      1
H-LC       XY25       94733.0      3
L-SC       OX40N      87339.0      1
H-LC       X200       76314.0      1
H-MC       X40N       75634.0      2
M-SC       X40        75054.0      1
H-MC       AR         59340.0      1
L-MC       XR         57091.0      1
H-SC       MH         55541.0      1
M-SC       OX40N      55374.0      2
L-LC       XY25       50283.0      1
H-LC       XY24       40738.0      1
H-MC       MH         32836.0      1
M-LC       XY25       17586.0      1
H-MC       OX40N      17279.0      1
H-LC       X5K        12626.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
